In [ ]:
import cv2 as cv
import mediapipe as mp
import numpy as np


hands = mp.solutions.hands.Hands(max_num_hands = 1)
draw = mp.solutions.drawing_utils

pencil_icon = cv.imread("pencil.png", cv.IMREAD_UNCHANGED)
eraser_icon = cv.imread("eraser.png", cv.IMREAD_UNCHANGED)
pencil_icon = cv.resize(pencil_icon, (30,30))
eraser_icon = cv.resize(eraser_icon, (30,30))

canvas = np.ones((480,640,3), np.uint8)*255
prev_canvas = (255, 255, 255)

p_x, p_y = 0, 0

clear_counter = 0
clear_threshold = 30

def nothing(x):
    pass

cv.namedWindow("Settings")
cv.resizeWindow("Settings", 640,480)

cv.createTrackbar("Pen R", "Settings", 0, 255, nothing)
cv.createTrackbar("Pen G", "Settings", 0, 255, nothing)
cv.createTrackbar("Pen B", "Settings", 0, 255, nothing)

cv.createTrackbar("Thickness", "Settings", 1, 50, nothing)

cv.createTrackbar("Canvas R", "Settings", 255, 255, nothing)
cv.createTrackbar("Canvas G", "Settings", 255, 255, nothing)
cv.createTrackbar("Canvas B", "Settings", 255, 255, nothing)

def is_pointing_up(hand_landmarks):
    lm = hand_landmarks.landmark
    index_up = lm[8].y < lm[6].y
    middle_down = lm[12].y > lm[10].y
    ring_down = lm[16].y > lm[14].y
    pinky_down = lm[20].y > lm[18].y
    return index_up and middle_down and ring_down and pinky_down

def is_two_finger(hand_landmarks):
    lm = hand_landmarks.landmark
    index_up = lm[8].y<lm[6].y
    middle_up = lm[12].y < lm[10].y
    ring_down = lm[16].y > lm[14].y
    pinky_down = lm[20].y > lm[18].y
    return index_up and middle_up and ring_down and pinky_down

def overlay_icon(frame, icon, x, y):
    ih, iw, ic = icon.shape
    bh,bw,_ = frame.shape
    if x < 0 or y < 0 or x + iw > bw or y + ih > bh:
        return
    if ic == 4:
        for c in range(3):
            frame[y : y + ih, x : x + iw, c] = (icon[:, :, c]*(icon[:, :, 3]/255) + frame[y : y + ih, x : x + iw, c]*(1 - icon[:, :, 3]/255)).astype('uint8')

#def nothing(x):
 #   pass

cap = cv.VideoCapture(0)
while True:
    r, frame = cap.read()
    frame = cv.flip(frame, 1)
    h, w, _ = frame.shape
    rgb = cv.cvtColor(frame, cv.COLOR_BGR2RGB)
    results = hands.process(rgb)
    
    r = cv.getTrackbarPos("Pen R", "Settings")
    g = cv.getTrackbarPos("Pen G", "Settings")
    b = cv.getTrackbarPos("Pen B", "Settings")
    thickness = cv.getTrackbarPos("Thickness", "Settings")

    c_r = cv.getTrackbarPos("Canvas R", "Settings")
    c_g = cv.getTrackbarPos("Canvas G", "Settings")
    c_b = cv.getTrackbarPos("Canvas B", "Settings")
    
    pen_color = (b,g,r)
    
    canvas_color = (c_r, c_g, c_b)
    
    eraser_color = canvas_color

    if canvas_color != prev_canvas:
        mask = np.all(canvas != prev_canvas, axis=2)
        canvas[:] = canvas_color
        canvas[mask] = (0, 0, 0)
        prev_canvas = canvas_color        
    
    if results.multi_hand_landmarks:
        for hand_landmarks in results.multi_hand_landmarks:
            landmark_list = hand_landmarks.landmark
            x1, y1 = int(landmark_list[8].x*w), int(landmark_list[8].y*h)
            x2, y2 = int(landmark_list[12].x*w), int(landmark_list[12].y*h)
            fingers = [landmark_list[8].y < landmark_list[6].y]
            
            if fingers[0]:
                if p_x == 0 and p_y == 0:
                    p_x, p_y = x1, y1
                cv.line(canvas, (p_x, p_y), (x1, y1), pen_color, thickness)
                p_x, p_y = x1, y1
            else:
                p_x, p_y = 0, 0

            if is_pointing_up(hand_landmarks):
                cx1 = x1 - 30//2
                cy1 = (y1 - 30//2)-20
                if cy1 >= 0 and cy1 + 50 <= frame.shape[0] and cx1 >= 0 and cx1 + 50 <= frame.shape[1]:
                    overlay_icon(frame, pencil_icon, cx1, cy1)

            if is_two_finger(hand_landmarks):
                cx2 = x2 - 30//2
                cy2 = (y2 - 30//2)-20
                if cy2 >= 0 and cy2 + 50 <= frame.shape[0] and cx2 >= 0 and cx2 + 50 <= frame.shape[1]:
                    overlay_icon(frame, eraser_icon, cx2, cy2)

            if landmark_list[8].y < landmark_list[6].y and landmark_list[12].y < landmark_list[10].y:
                cv.circle(canvas, (x1,y1), 20, eraser_color, -1)

            def open_palm(hand_landmarks):
                tips = [4, 8, 12, 16, 20]
                dips = [3, 6, 10, 14, 18]
                palm = []
                for tip,dip in zip(tips, dips):
                    palm.append(hand_landmarks.landmark[tip].y < hand_landmarks.landmark[dip].y)
                return palm

            palm = open_palm(hand_landmarks)
            if sum(palm) == 5:
                clear_counter +=1
                if clear_counter > clear_threshold:
                    canvas[:] = canvas_color
                    p_x, p_y = 0, 0

    combined = np.hstack((frame, canvas))
    
    cv.imshow("Settings", combined)
    key = cv.waitKey(1) & 0xff
    if key == ord('s'):
        cv.imwrite("drawing.png", canvas)
    elif key == ord('z'):
        break

cap.release()
cv.destroyAllWindows()